In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#from src.data.data_collector import DataCollector
from src.models.model_trainer_rl_v2 import ModelTrainerRL, TradingEnvRL
from src.models.backtester import PortfolioBacktester, PortfolioBacktesterRL
from src.utils.config_loader import load_config

In [21]:
# data = pd.read_csv('data/processed/AAPL_processed.csv')
config = load_config("config/config.yaml")

In [24]:
# # --- CONFIG OVERRIDES (Notebook-level, no class changes) ---
# # Adjust reinforcement learning hyperparameters here without changing class code.

# def set_rl_params(cfg, algo: str, **kwargs):
#     rl = cfg.setdefault('reinforcement_learning', {})
#     algo_cfg = rl.setdefault(algo, {})
#     algo_cfg.update(kwargs)

# # Example: Increase SAC training steps
# set_rl_params(config, 'sac', total_timesteps=300_000)

# # Optional: tweak other algos similarly
# # set_rl_params(config, 'ppo', total_timesteps=300_000)
# # set_rl_params(config, 'td3', total_timesteps=300_000)

# print("SAC total_timesteps:", config['reinforcement_learning']['sac']['total_timesteps'])

In [25]:
# data['date'] = pd.to_datetime(data['date'])
# data.set_index('date', inplace=True)

## PPO Agent

In [ ]:
# Load Data
# (Ensure your CSV has columns like 'close', 'rsi', 'sma', etc.)
try:
    data = pd.read_csv('data/processed/AAPL_processed.csv')
    if 'Date' in data.columns:
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: Data file not found. Check path.")

# Split Train/Test
split_idx = int(len(data) * 0.7)
train_df = data.iloc[:split_idx]
test_df = data.iloc[split_idx:]

# Initialize Trainer
trainer = ModelTrainerRL(config['reinforcement_learning'])

print("Training PPO Agent...")


env_params = config['reinforcement_learning']['environment']

env_train = TradingEnvRL(
    train_df, 
    initial_balance=env_params.get('initial_balance', 10000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

result = trainer.train_ppo(env_train)
model = result['model'] 

print("Training Complete.")

print("Generating Agent Predictions on Test Data...")


env_params = config['reinforcement_learning']['environment']


env_test = TradingEnvRL(
    test_df, 
    initial_balance=env_params.get('initial_balance', 100000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

obs, _ = env_test.reset()
done = False
actions = []

# Manual Inference Loop
while not done:
    action, _ = model.predict(obs, deterministic=True)
    actions.append(action[0]) 
    obs, _, terminated, truncated, _ = env_test.step(action)
    done = terminated or truncated


#  RUN BACKTEST ---
print("Running Backtest...")


backtester = PortfolioBacktesterRL(env_params) 

portfolio = backtester.run_backtest(
    price_data=test_df['close'], 
    predicted_weights=np.array(actions),
    lookback_window=env_params.get('lookback_window', 30) # Important for data alignment
)
comparison = backtester.compare_with_buy_and_hold_rl()
# Show Metrics
metrics = backtester.get_performance_metrics()
print("\n--- Strategy Performance ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Plot
portfolio.plot().show()

C:\Users\BOOKLAPTOP\AppData\Local\Temp\ipykernel_45752\2155576757.py:6: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



Data loaded successfully.
Training PPO Agent...


d:\MSDS\buy-sell-hold-strategy-prediction\venv-3.11\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning:

In the future `np.object` will be defined as the corresponding NumPy scalar.

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
INFO:src.models.model_trainer_rl_v2:Training PPO for 200000 timesteps
INFO:BacktesterRL:Preparing Backtest. Raw Prices: 488, Predictions: 427
INFO:BacktesterRL:Running vectorbt simulation...


Training Complete.
Generating Agent Predictions on Test Data...
Running Backtest...


INFO:BacktesterRL:Backtest successfully completed.
INFO:BacktesterRL:Strategy Return: 46.15%
INFO:BacktesterRL:Buy & Hold Return: 54.80%
INFO:BacktesterRL:Outperformance: -8.65%



--- Strategy Performance ---
Total Return (%): 46.1500
Annual Return (%): 38.3200
Sharpe Ratio: 1.1142
Sortino Ratio: 1.7076
Max Drawdown (%): -38.3500
Calmar Ratio: 0.9991
Win Rate (%): 71.4300
Total Trades: 21.0000
Final Value ($): 146150.0900


In [8]:
# Load Data
# (Ensure your CSV has columns like 'close', 'rsi', 'sma', etc.)
try:
    data = pd.read_csv('data/processed/AAPL_processed.csv')
    if 'Date' in data.columns:
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: Data file not found. Check path.")

# Split Train/Test
split_idx = int(len(data) * 0.7)
train_df = data.iloc[:split_idx]
test_df = data.iloc[split_idx:]

# Initialize Trainer
trainer = ModelTrainerRL(config['reinforcement_learning'])

print("Training PPO Agent...")


env_params = config['reinforcement_learning']['environment']

env_train = TradingEnvRL(
    train_df, 
    initial_balance=env_params.get('initial_balance', 10000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

result = trainer.train_ppo(env_train)
model = result['model'] 

print("Training Complete.")

print("Generating Agent Predictions on Test Data...")


env_params = config['reinforcement_learning']['environment']


env_test = TradingEnvRL(
    test_df, 
    initial_balance=env_params.get('initial_balance', 100000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

obs, _ = env_test.reset()
done = False
actions = []

# Manual Inference Loop
while not done:
    action, _ = model.predict(obs, deterministic=True)
    actions.append(action[0]) 
    obs, _, terminated, truncated, _ = env_test.step(action)
    done = terminated or truncated


#  RUN BACKTEST ---
print("Running Backtest...")


backtester = PortfolioBacktesterRL(env_params) 

portfolio = backtester.run_backtest(
    price_data=test_df['close'], 
    predicted_weights=np.array(actions),
    lookback_window=env_params.get('lookback_window', 30) # Important for data alignment
)
comparison = backtester.compare_with_buy_and_hold_rl()
# Show Metrics
metrics = backtester.get_performance_metrics()
print("\n--- Strategy Performance ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Plot
portfolio.plot().show()

C:\Users\BOOKLAPTOP\AppData\Local\Temp\ipykernel_45752\3030507848.py:6: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`

INFO:src.models.model_trainer_rl_v2:Training PPO for 200000 timesteps


Data loaded successfully.
Training PPO Agent...


INFO:BacktesterRL:Preparing Backtest. Raw Prices: 732, Predictions: 671
INFO:BacktesterRL:Running vectorbt simulation...
INFO:BacktesterRL:Backtest successfully completed.
INFO:BacktesterRL:Strategy Return: 59.75%
INFO:BacktesterRL:Buy & Hold Return: 70.81%
INFO:BacktesterRL:Outperformance: -11.06%


Training Complete.
Generating Agent Predictions on Test Data...
Running Backtest...

--- Strategy Performance ---
Total Return (%): 59.7500
Annual Return (%): 29.0200
Sharpe Ratio: 0.9785
Sortino Ratio: 1.4757
Max Drawdown (%): -34.0600
Calmar Ratio: 0.8520
Win Rate (%): 84.6200
Total Trades: 26.0000
Final Value ($): 159746.7500


TSLA

In [5]:
# Load Data
# (Ensure your CSV has columns like 'close', 'rsi', 'sma', etc.)
try:
    data = pd.read_csv('data/processed/TSLA_processed.csv')
    if 'Date' in data.columns:
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: Data file not found. Check path.")

# Split Train/Test
split_idx = int(len(data) * 0.8)
train_df = data.iloc[:split_idx]
test_df = data.iloc[split_idx:]

# Initialize Trainer
trainer = ModelTrainerRL(config['reinforcement_learning'])

print("Training PPO Agent...")


env_params = config['reinforcement_learning']['environment']

env_train = TradingEnvRL(
    train_df, 
    initial_balance=env_params.get('initial_balance', 10000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

result = trainer.train_ppo(env_train)
model = result['model'] 

print("Training Complete.")

print("Generating Agent Predictions on Test Data...")


env_params = config['reinforcement_learning']['environment']


env_test = TradingEnvRL(
    test_df, 
    initial_balance=env_params.get('initial_balance', 100000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

obs, _ = env_test.reset()
done = False
actions = []

# Manual Inference Loop
while not done:
    action, _ = model.predict(obs, deterministic=True)
    actions.append(action[0]) 
    obs, _, terminated, truncated, _ = env_test.step(action)
    done = terminated or truncated


#  RUN BACKTEST ---
print("Running Backtest...")


backtester = PortfolioBacktesterRL(env_params) 

portfolio = backtester.run_backtest(
    price_data=test_df['close'], 
    predicted_weights=np.array(actions),
    lookback_window=env_params.get('lookback_window', 30) # Important for data alignment
)
comparison = backtester.compare_with_buy_and_hold_rl()
# Show Metrics
metrics = backtester.get_performance_metrics()
print("\n--- Strategy Performance ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Plot
portfolio.plot().show()

C:\Users\BOOKLAPTOP\AppData\Local\Temp\ipykernel_45752\1140079795.py:6: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`

INFO:src.models.model_trainer_rl_v2:Training PPO for 200000 timesteps


Data loaded successfully.
Training PPO Agent...


INFO:BacktesterRL:Preparing Backtest. Raw Prices: 484, Predictions: 423
INFO:BacktesterRL:Running vectorbt simulation...


Training Complete.
Generating Agent Predictions on Test Data...
Running Backtest...


INFO:BacktesterRL:Backtest successfully completed.
INFO:BacktesterRL:Strategy Return: -57.64%
INFO:BacktesterRL:Buy & Hold Return: 124.32%
INFO:BacktesterRL:Outperformance: -181.96%



--- Strategy Performance ---
Total Return (%): -57.6400
Annual Return (%): -52.3500
Sharpe Ratio: -0.6949
Sortino Ratio: -0.9207
Max Drawdown (%): -75.6600
Calmar Ratio: -0.6919
Win Rate (%): 35.6500
Total Trades: 216.0000
Final Value ($): 42358.2800


AMZN

In [10]:
# Load Data
# (Ensure your CSV has columns like 'close', 'rsi', 'sma', etc.)
try:
    data = pd.read_csv('data/processed/CWCO_processed.csv')
    if 'Date' in data.columns:
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: Data file not found. Check path.")

# Split Train/Test
split_idx = int(len(data) * 0.7)
train_df = data.iloc[:split_idx]
test_df = data.iloc[split_idx:]

# Initialize Trainer
trainer = ModelTrainerRL(config['reinforcement_learning'])

print("Training PPO Agent...")


env_params = config['reinforcement_learning']['environment']

env_train = TradingEnvRL(
    train_df, 
    initial_balance=env_params.get('initial_balance', 10000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

result = trainer.train_ppo(env_train)
model = result['model'] 

print("Training Complete.")

print("Generating Agent Predictions on Test Data...")


env_params = config['reinforcement_learning']['environment']


env_test = TradingEnvRL(
    test_df, 
    initial_balance=env_params.get('initial_balance', 100000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

obs, _ = env_test.reset()
done = False
actions = []

# Manual Inference Loop
while not done:
    action, _ = model.predict(obs, deterministic=True)
    actions.append(action[0]) 
    obs, _, terminated, truncated, _ = env_test.step(action)
    done = terminated or truncated


#  RUN BACKTEST ---
print("Running Backtest...")


backtester = PortfolioBacktesterRL(env_params) 

portfolio = backtester.run_backtest(
    price_data=test_df['close'], 
    predicted_weights=np.array(actions),
    lookback_window=env_params.get('lookback_window', 30) # Important for data alignment
)
comparison = backtester.compare_with_buy_and_hold_rl()
# Show Metrics
metrics = backtester.get_performance_metrics()
print("\n--- Strategy Performance ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Plot
portfolio.plot().show()

C:\Users\BOOKLAPTOP\AppData\Local\Temp\ipykernel_45752\4278082664.py:6: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



Data loaded successfully.
Training PPO Agent...


INFO:src.models.model_trainer_rl_v2:Training PPO for 200000 timesteps
INFO:BacktesterRL:Preparing Backtest. Raw Prices: 732, Predictions: 671
INFO:BacktesterRL:Running vectorbt simulation...


Training Complete.
Generating Agent Predictions on Test Data...
Running Backtest...


INFO:BacktesterRL:Backtest successfully completed.
INFO:BacktesterRL:Strategy Return: 63.77%
INFO:BacktesterRL:Buy & Hold Return: 144.10%
INFO:BacktesterRL:Outperformance: -80.34%



--- Strategy Performance ---
Total Return (%): 63.7700
Annual Return (%): 30.7800
Sharpe Ratio: 1.2884
Sortino Ratio: 2.0382
Max Drawdown (%): -20.7600
Calmar Ratio: 1.4826
Win Rate (%): 44.6700
Total Trades: 347.0000
Final Value ($): 163765.1900


In [12]:
## 2020-2025 

# Load Data
# (Ensure your CSV has columns like 'close', 'rsi', 'sma', etc.)
try:
    data = pd.read_csv('data/processed/CWCO_processed_2024.csv')
    if 'Date' in data.columns:
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: Data file not found. Check path.")

# Split Train/Test
split_idx = int(len(data) * 0.7)
train_df = data.iloc[:split_idx]
test_df = data.iloc[split_idx:]

# Initialize Trainer
trainer = ModelTrainerRL(config['reinforcement_learning'])

print("Training PPO Agent...")


env_params = config['reinforcement_learning']['environment']

env_train = TradingEnvRL(
    train_df, 
    initial_balance=env_params.get('initial_balance', 10000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

result = trainer.train_ppo(env_train)
model = result['model'] 

print("Training Complete.")

print("Generating Agent Predictions on Test Data...")


env_params = config['reinforcement_learning']['environment']


env_test = TradingEnvRL(
    test_df, 
    initial_balance=env_params.get('initial_balance', 100000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

obs, _ = env_test.reset()
done = False
actions = []

# Manual Inference Loop
while not done:
    action, _ = model.predict(obs, deterministic=True)
    actions.append(action[0]) 
    obs, _, terminated, truncated, _ = env_test.step(action)
    done = terminated or truncated


#  RUN BACKTEST ---
print("Running Backtest...")


backtester = PortfolioBacktesterRL(env_params) 

portfolio = backtester.run_backtest(
    price_data=test_df['close'], 
    predicted_weights=np.array(actions),
    lookback_window=env_params.get('lookback_window', 30) # Important for data alignment
)
comparison = backtester.compare_with_buy_and_hold_rl()
# Show Metrics
metrics = backtester.get_performance_metrics()
print("\n--- Strategy Performance ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Plot
portfolio.plot().show()

C:\Users\BOOKLAPTOP\AppData\Local\Temp\ipykernel_45752\3918487927.py:8: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`

INFO:src.models.model_trainer_rl_v2:Training PPO for 200000 timesteps


Data loaded successfully.
Training PPO Agent...


INFO:BacktesterRL:Preparing Backtest. Raw Prices: 128, Predictions: 67
INFO:BacktesterRL:Running vectorbt simulation...
INFO:BacktesterRL:Backtest successfully completed.
INFO:BacktesterRL:Strategy Return: 2.62%
INFO:BacktesterRL:Buy & Hold Return: 2.62%
INFO:BacktesterRL:Outperformance: 0.00%


Training Complete.
Generating Agent Predictions on Test Data...
Running Backtest...

--- Strategy Performance ---
Total Return (%): 2.6200
Annual Return (%): 15.1300
Sharpe Ratio: 0.5768
Sortino Ratio: 0.9198
Max Drawdown (%): -10.9300
Calmar Ratio: 1.3836
Win Rate (%): 100.0000
Total Trades: 1.0000
Final Value ($): 102619.8200


In [ ]:
#drop open high low volume future_return
train_df = train_df.drop(columns=['open', 'high', 'low', 'volume', 'future_return'], errors='ignore')

,open,high,low,close,volume,dividends,stock splits,sma_20,sma_50,rsi_14,...,rolling_low_52w,fibo_23.6,fibo_38.2,fibo_50.0,fibo_61.8,fibo_78.6,fibo_100.0,fibo_161.8,future_return,label
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-15 00:00:00-04:00,9.524449,9.851268,9.524449,9.664515,39600,0.0,0.0,9.383916,9.404552,53.307158,...,8.413262,10.104019,9.780916,9.519778,9.258640,8.886851,8.413262,7.045608,-0.014493,1
2022-03-16 00:00:00-04:00,9.571136,9.981994,9.337693,9.496434,123500,0.0,0.0,9.397922,9.393907,50.073807,...,8.413262,10.104019,9.780916,9.519778,9.258640,8.886851,8.413262,7.045608,0.010816,1
2022-03-17 00:00:00-04:00,9.617826,9.823255,9.608488,9.617826,75700,0.0,0.0,9.418932,9.384943,52.323024,...,8.413262,10.104019,9.780916,9.519778,9.258640,8.886851,8.413262,7.045608,-0.010680,1
2022-03-18 00:00:00-04:00,9.664513,9.692526,9.094913,9.094913,795600,0.0,0.0,9.414730,9.367762,43.278240,...,8.413262,10.104019,9.780916,9.519778,9.258640,8.886851,8.413262,7.045608,0.067762,2
2022-03-21 00:00:00-04:00,9.262993,9.412396,9.094914,9.356370,190500,0.0,0.0,9.430604,9.362719,48.108363,...,8.413262,10.104019,9.780916,9.519778,9.258640,8.886851,8.413262,7.045608,0.037924,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-04 00:00:00-04:00,24.706096,24.725751,24.460411,24.558685,44700,0.0,0.0,24.865740,25.916989,43.808114,...,23.106317,33.809334,31.763993,30.110909,28.457825,26.104282,23.106317,14.448641,0.008803,1
2024-10-07 00:00:00-04:00,24.509547,24.784713,24.106623,24.371962,83300,0.0,0.0,24.860293,25.827337,42.033247,...,23.106317,33.809334,31.763993,30.110909,28.457825,26.104282,23.106317,14.448641,0.043952,2
2024-10-08 00:00:00-04:00,24.480067,24.480067,23.831458,24.185246,87100,0.0,0.0,24.816647,25.756063,40.276032,...,23.106317,33.809334,31.763993,30.110909,28.457825,26.104282,23.106317,14.448641,0.056887,2


In [ ]:
## 2020-2025 

# Load Data
# (Ensure your CSV has columns like 'close', 'rsi', 'sma', etc.)
try:
    data = pd.read_csv('data/processed/CWCO_processed_2022.csv')
    if 'Date' in data.columns:
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: Data file not found. Check path.")

# Split Train/Test
split_idx = int(len(data) * 0.7)
train_df = data.iloc[:split_idx]
#train_df = train_df.drop(columns=['open', 'high', 'low', 'volume', 'future_return'], errors='ignore')
test_df = data.iloc[split_idx:]
#test_df = test_df.drop(columns=['open', 'high', 'low', 'volume', 'future_return'], errors='ignore')

# Initialize Trainer
trainer = ModelTrainerRL(config['reinforcement_learning'])

print("Training PPO Agent...")


env_params = config['reinforcement_learning']['environment']

env_train = TradingEnvRL(
    train_df, 
    initial_balance=env_params.get('initial_balance', 10000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

result = trainer.train_ppo(env_train)
model = result['model'] 

print("Training Complete.")

print("Generating Agent Predictions on Test Data...")


env_params = config['reinforcement_learning']['environment']


env_test = TradingEnvRL(
    test_df, 
    initial_balance=env_params.get('initial_balance', 100000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

obs, _ = env_test.reset()
done = False
actions = []

# Manual Inference Loop
while not done:
    action, _ = model.predict(obs, deterministic=True)
    actions.append(action[0]) 
    obs, _, terminated, truncated, _ = env_test.step(action)
    done = terminated or truncated


#  RUN BACKTEST ---
print("Running Backtest...")


backtester = PortfolioBacktesterRL(env_params) 

portfolio = backtester.run_backtest(
    price_data=test_df['close'], 
    predicted_weights=np.array(actions),
    lookback_window=env_params.get('lookback_window', 30) # Important for data alignment
)
comparison = backtester.compare_with_buy_and_hold_rl()
# Show Metrics
metrics = backtester.get_performance_metrics()
print("\n--- Strategy Performance ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Plot
portfolio.plot().show()

C:\Users\BOOKLAPTOP\AppData\Local\Temp\ipykernel_45752\2735129743.py:8: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`

INFO:src.models.model_trainer_rl_v2:Training PPO for 150000 timesteps


Data loaded successfully.
Training PPO Agent...


INFO:BacktesterRL:Preparing Backtest. Raw Prices: 279, Predictions: 218
INFO:BacktesterRL:Running vectorbt simulation...
INFO:BacktesterRL:Backtest successfully completed.
INFO:BacktesterRL:Strategy Return: 3.10%
INFO:BacktesterRL:Buy & Hold Return: 32.68%
INFO:BacktesterRL:Outperformance: -29.58%


Training Complete.
Generating Agent Predictions on Test Data...
Running Backtest...

--- Strategy Performance ---
Total Return (%): 3.1000
Annual Return (%): 5.2500
Sharpe Ratio: 1.4270
Sortino Ratio: 2.1382
Max Drawdown (%): -2.2700
Calmar Ratio: 2.3115
Win Rate (%): 81.2000
Total Trades: 117.0000
Final Value ($): 103103.5600


In [22]:
## 2020-2025 

# Load Data
# (Ensure your CSV has columns like 'close', 'rsi', 'sma', etc.)
try:
    data = pd.read_csv('data/processed/CWCO_processed.csv')
    if 'Date' in data.columns:
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: Data file not found. Check path.")

# Split Train/Test
split_idx = int(len(data) * 0.7)
train_df = data.iloc[:split_idx]
#train_df = train_df.drop(columns=['open', 'high', 'low', 'volume', 'future_return'], errors='ignore')
test_df = data.iloc[split_idx:]
#test_df = test_df.drop(columns=['open', 'high', 'low', 'volume', 'future_return'], errors='ignore')

# Initialize Trainer
trainer = ModelTrainerRL(config['reinforcement_learning'])

print("Training PPO Agent...")


env_params = config['reinforcement_learning']['environment']

env_train = TradingEnvRL(
    train_df, 
    initial_balance=env_params.get('initial_balance', 10000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

result = trainer.train_ppo(env_train)
model = result['model'] 

print("Training Complete.")

print("Generating Agent Predictions on Test Data...")


env_params = config['reinforcement_learning']['environment']


env_test = TradingEnvRL(
    test_df, 
    initial_balance=env_params.get('initial_balance', 100000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

obs, _ = env_test.reset()
done = False
actions = []

# Manual Inference Loop
while not done:
    action, _ = model.predict(obs, deterministic=True)
    actions.append(action[0]) 
    obs, _, terminated, truncated, _ = env_test.step(action)
    done = terminated or truncated


#  RUN BACKTEST ---
print("Running Backtest...")


backtester = PortfolioBacktesterRL(env_params) 

portfolio = backtester.run_backtest(
    price_data=test_df['close'], 
    predicted_weights=np.array(actions),
    lookback_window=env_params.get('lookback_window', 30) # Important for data alignment
)
comparison = backtester.compare_with_buy_and_hold_rl()
# Show Metrics
metrics = backtester.get_performance_metrics()
print("\n--- Strategy Performance ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Plot
portfolio.plot().show()

C:\Users\BOOKLAPTOP\AppData\Local\Temp\ipykernel_45752\1028939540.py:8: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`

INFO:src.models.model_trainer_rl_v2:Training PPO for 150000 timesteps


Data loaded successfully.
Training PPO Agent...
Training Complete.
Generating Agent Predictions on Test Data...


INFO:BacktesterRL:Preparing Backtest. Raw Prices: 732, Predictions: 671
INFO:BacktesterRL:Running vectorbt simulation...
INFO:BacktesterRL:Backtest successfully completed.
INFO:BacktesterRL:Strategy Return: -72.14%
INFO:BacktesterRL:Buy & Hold Return: 144.10%
INFO:BacktesterRL:Outperformance: -216.24%


Running Backtest...

--- Strategy Performance ---
Total Return (%): -72.1400
Annual Return (%): -50.1000
Sharpe Ratio: -1.3459
Sortino Ratio: -1.7300
Max Drawdown (%): -75.5900
Calmar Ratio: -0.6628
Win Rate (%): 25.2900
Total Trades: 348.0000
Final Value ($): 27862.2800


In [23]:
# SPY

# Load Data
# (Ensure your CSV has columns like 'close', 'rsi', 'sma', etc.)
try:
    data = pd.read_csv('data/processed/SPY_processed.csv')
    if 'Date' in data.columns:
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: Data file not found. Check path.")

# Split Train/Test
split_idx = int(len(data) * 0.7)
train_df = data.iloc[:split_idx]
#train_df = train_df.drop(columns=['open', 'high', 'low', 'volume', 'future_return'], errors='ignore')
test_df = data.iloc[split_idx:]
#test_df = test_df.drop(columns=['open', 'high', 'low', 'volume', 'future_return'], errors='ignore')

# Initialize Trainer
trainer = ModelTrainerRL(config['reinforcement_learning'])

print("Training PPO Agent...")


env_params = config['reinforcement_learning']['environment']

env_train = TradingEnvRL(
    train_df, 
    initial_balance=env_params.get('initial_balance', 10000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

result = trainer.train_ppo(env_train)
model = result['model'] 

print("Training Complete.")

print("Generating Agent Predictions on Test Data...")


env_params = config['reinforcement_learning']['environment']


env_test = TradingEnvRL(
    test_df, 
    initial_balance=env_params.get('initial_balance', 100000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

obs, _ = env_test.reset()
done = False
actions = []

# Manual Inference Loop
while not done:
    action, _ = model.predict(obs, deterministic=True)
    actions.append(action[0]) 
    obs, _, terminated, truncated, _ = env_test.step(action)
    done = terminated or truncated


#  RUN BACKTEST ---
print("Running Backtest...")


backtester = PortfolioBacktesterRL(env_params) 

portfolio = backtester.run_backtest(
    price_data=test_df['close'], 
    predicted_weights=np.array(actions),
    lookback_window=env_params.get('lookback_window', 30) # Important for data alignment
)
comparison = backtester.compare_with_buy_and_hold_rl()
# Show Metrics
metrics = backtester.get_performance_metrics()
print("\n--- Strategy Performance ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Plot
portfolio.plot().show()

C:\Users\BOOKLAPTOP\AppData\Local\Temp\ipykernel_45752\1733773946.py:8: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`

INFO:src.models.model_trainer_rl_v2:Training PPO for 150000 timesteps


Data loaded successfully.
Training PPO Agent...
Training Complete.
Generating Agent Predictions on Test Data...


INFO:BacktesterRL:Preparing Backtest. Raw Prices: 1939, Predictions: 1000
INFO:BacktesterRL:Running vectorbt simulation...
INFO:BacktesterRL:Backtest successfully completed.
INFO:BacktesterRL:Strategy Return: -40.19%
INFO:BacktesterRL:Buy & Hold Return: 48.41%
INFO:BacktesterRL:Outperformance: -88.61%


Running Backtest...

--- Strategy Performance ---
Total Return (%): -40.1900
Annual Return (%): -17.1100
Sharpe Ratio: -0.7860
Sortino Ratio: -1.0818
Max Drawdown (%): -53.7000
Calmar Ratio: -0.3186
Win Rate (%): 26.6300
Total Trades: 537.0000
Final Value ($): 59806.4100


In [24]:
# Load Data
# (Ensure your CSV has columns like 'close', 'rsi', 'sma', etc.)
try:
    data = pd.read_csv('data/processed/BAC_processed.csv')
    if 'Datetime' in data.columns:
        data['Datetime'] = pd.to_datetime(data['Datetime'])
        data.set_index('Datetime', inplace=True)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: Data file not found. Check path.")

# Split Train/Test
split_idx = int(len(data) * 0.7)
train_df = data.iloc[:split_idx]
test_df = data.iloc[split_idx:]

# Initialize Trainer
trainer = ModelTrainerRL(config['reinforcement_learning'])

print("Training PPO Agent...")


env_params = config['reinforcement_learning']['environment']

env_train = TradingEnvRL(
    train_df, 
    initial_balance=env_params.get('initial_balance', 10000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

result = trainer.train_ppo(env_train)
model = result['model'] 

print("Training Complete.")

print("Generating Agent Predictions on Test Data...")


env_params = config['reinforcement_learning']['environment']


env_test = TradingEnvRL(
    test_df, 
    initial_balance=env_params.get('initial_balance', 100000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

obs, _ = env_test.reset()
done = False
actions = []

# Manual Inference Loop
while not done:
    action, _ = model.predict(obs, deterministic=True)
    actions.append(action[0]) 
    obs, _, terminated, truncated, _ = env_test.step(action)
    done = terminated or truncated


#  RUN BACKTEST ---
print("Running Backtest...")


backtester = PortfolioBacktesterRL(env_params) 

portfolio = backtester.run_backtest(
    price_data=test_df['close'], 
    predicted_weights=np.array(actions),
    lookback_window=env_params.get('lookback_window', 30) # Important for data alignment
)
comparison = backtester.compare_with_buy_and_hold_rl()
# Show Metrics
metrics = backtester.get_performance_metrics()
print("\n--- Strategy Performance ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Plot
portfolio.plot().show()

Data loaded successfully.
Training PPO Agent...


ValueError: could not convert string to float: '2015-03-16 00:00:00-04:00'

## SAC

In [27]:
try:
    data = pd.read_csv('data/processed/AAPL_processed.csv')
    if 'Date' in data.columns:
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: Data file not found. Check path.")

# Split Train/Test
split_idx = int(len(data) * 0.8)
train_df = data.iloc[:split_idx]
test_df = data.iloc[split_idx:]

# Initialize Trainer
trainer = ModelTrainerRL(config['reinforcement_learning'])

print("Training SAC Agent...")


env_params = config['reinforcement_learning']['environment']

env_train = TradingEnvRL(
    train_df, 
    initial_balance=env_params.get('initial_balance', 10000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

result = trainer.train_sac(env_train)
model = result['model'] 

print("Training Complete.")

print("Generating Agent Predictions on Test Data...")

# Unpack the config dictionary using **
env_params = config['reinforcement_learning']['environment']

env_test = TradingEnvRL(
    test_df, 
    initial_balance=env_params.get('initial_balance', 10000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

obs, _ = env_test.reset()
done = False
actions = []

# Manual Inference Loop
while not done:
    action, _ = model.predict(obs, deterministic=True)
    actions.append(action[0]) 
    obs, _, terminated, truncated, _ = env_test.step(action)
    done = terminated or truncated


# --- CELL 5: RUN BACKTEST ---
print("Running Backtest...")

# FIX: Use the correct config path
# We pass the same env_params we extracted in Cell 4
backtester = PortfolioBacktesterRL(env_params) 

portfolio = backtester.run_backtest(
    price_data=test_df['close'], 
    predicted_weights=np.array(actions),
    lookback_window=env_params.get('lookback_window', 30) # Important for data alignment
)
comparison = backtester.compare_with_buy_and_hold_rl()
# Show Metrics
metrics = backtester.get_performance_metrics()
print("\n--- Strategy Performance ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Plot
portfolio.plot().show()

C:\Users\BOOKLAPTOP\AppData\Local\Temp\ipykernel_24508\4234177624.py:4: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`

INFO:src.models.model_trainer_rl_v2:Training SAC for 5000 timesteps


Data loaded successfully.
Training SAC Agent...


INFO:BacktesterRL:Preparing Backtest. Raw Prices: 488, Predictions: 457
INFO:BacktesterRL:Running vectorbt simulation...
INFO:BacktesterRL:Backtest successfully completed.
INFO:BacktesterRL:Strategy Return: -39.37%
INFO:BacktesterRL:Buy & Hold Return: 40.04%
INFO:BacktesterRL:Outperformance: -79.41%


Training Complete.
Generating Agent Predictions on Test Data...
Running Backtest...

--- Strategy Performance ---
Total Return (%): -39.3700
Annual Return (%): -32.9400
Sharpe Ratio: -0.9773
Sortino Ratio: -1.2955
Max Drawdown (%): -48.8100
Calmar Ratio: -0.6750
Win Rate (%): 21.7400
Total Trades: 253.0000
Final Value ($): 6062.9600


## TD3

In [28]:
# Load Data
# (Ensure your CSV has columns like 'close', 'rsi', 'sma', etc.)
try:
    data = pd.read_csv('data/processed/AAPL_processed.csv')
    if 'Date' in data.columns:
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: Data file not found. Check path.")

# --- CELL 3: TRAIN MODEL ---
# Split Train/Test
split_idx = int(len(data) * 0.8)
train_df = data.iloc[:split_idx]
test_df = data.iloc[split_idx:]

# Initialize Trainer
trainer = ModelTrainerRL(config['reinforcement_learning'])

print("Training TD3 Agent...")

# FIX 1: Instantiate the Environment manually
env_params = config['reinforcement_learning']['environment']

env_train = TradingEnvRL(
    train_df, 
    initial_balance=env_params.get('initial_balance', 10000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)
# FIX 2: Pass the ENV (not the DF) to the trainer
# FIX 3: The old class returns a DICT, so we must extract 'model'
result = trainer.train_td3(env_train)
model = result['model'] 

print("Training Complete.")

# --- CELL 4: GENERATE PREDICTIONS (THE BRIDGE) ---
print("Generating Agent Predictions on Test Data...")

# FIX: Unpack the config dictionary using **
env_params = config['reinforcement_learning']['environment']

# Explicitly passing arguments is safer than ** unpacking in notebooks
env_test = TradingEnvRL(
    test_df, 
    initial_balance=env_params.get('initial_balance', 10000),
    commission=env_params.get('commission', 0.001),
    lookback_window=env_params.get('lookback_window', 30)
)

obs, _ = env_test.reset()
done = False
actions = []

# Manual Inference Loop
while not done:
    action, _ = model.predict(obs, deterministic=True)
    actions.append(action[0]) 
    obs, _, terminated, truncated, _ = env_test.step(action)
    done = terminated or truncated


# --- CELL 5: RUN BACKTEST ---
print("Running Backtest...")

# FIX: Use the correct config path
# We pass the same env_params we extracted in Cell 4
backtester = PortfolioBacktesterRL(env_params) 

portfolio = backtester.run_backtest(
    price_data=test_df['close'], 
    predicted_weights=np.array(actions),
    lookback_window=env_params.get('lookback_window', 30) # Important for data alignment
)
comparison = backtester.compare_with_buy_and_hold_rl()
# Show Metrics
metrics = backtester.get_performance_metrics()
print("\n--- Strategy Performance ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Plot
portfolio.plot().show()

C:\Users\BOOKLAPTOP\AppData\Local\Temp\ipykernel_24508\3201286003.py:6: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`

INFO:src.models.model_trainer_rl_v2:Training TD3 for 5000 timesteps


Data loaded successfully.
Training TD3 Agent...


INFO:BacktesterRL:Preparing Backtest. Raw Prices: 488, Predictions: 457
INFO:BacktesterRL:Running vectorbt simulation...
INFO:BacktesterRL:Backtest successfully completed.
INFO:BacktesterRL:Strategy Return: -39.37%
INFO:BacktesterRL:Buy & Hold Return: 40.04%
INFO:BacktesterRL:Outperformance: -79.41%


Training Complete.
Generating Agent Predictions on Test Data...
Running Backtest...

--- Strategy Performance ---
Total Return (%): -39.3700
Annual Return (%): -32.9400
Sharpe Ratio: -0.9773
Sortino Ratio: -1.2955
Max Drawdown (%): -48.8100
Calmar Ratio: -0.6750
Win Rate (%): 21.7400
Total Trades: 253.0000
Final Value ($): 6062.9600
